# Extracting Time-series Spectra from Raw Spectrograph Images

This document outlines the approach and tools needed to go from raw spectrograph science and calibration images to wavelength-calibrated, time-series spectra for multiple stars. 

## Motivation

Let design something that is built on `astropy` (including possibly `ccdproc` and `specreduce`), easy-to-use, pedagogical, testable, and friendly.
 
- We want to go from observations to spectra and to be able to explain easily to another scientist how we did it.
- We want to be able to automate, but we don't want a mysterious magical box into which we just throw our data.
- We want to be able to analyze data from multiple telescopes using a similar toolkit. 
- We want documentation that is fun to read and easy to understand.
- We want a tool that can be installed via a single pip command in almost any up-to-date Python environment.
- We want to support the development of more `astropy`-centered tools, for repeatability, interoperability, and community support.

## General Approach 

 Let's aim for the dependencies to be standard tools, `astropy`, and `astropy`-coordinated packages. If there's something they're missing, let's be good community members and try to help them add it! The core tools I suspect we'll need include:

 - [astropy](https://docs.astropy.org/en/stable/)
 - [ccdproc](https://ccdproc.readthedocs.io/en/latest/)
 - [photutils](https://photutils.readthedocs.io/en/stable/)
 - [specutils](https://specutils.readthedocs.io/en/stable/)
 - [specreduce](https://specreduce.readthedocs.io/en/latest/)
 
We should particularly prioritize useful visualizations, careful treatment of units, the ability to automatically test as much of the code as possible, and carefully measuring lots of diagnostics to help time-series systematics analyses.

We might also draw conceptual ideas from the following, citing them where appropriate.
- [IRAF](https://iraf-community.github.io/) = the classic 
- [PyKOSMOS](https://github.com/jradavenport/pykosmos) = a mostly workable but not super duper yet clearly documented tool for KOSMOS spectral data
- [PypeIt](https://github.com/pypeit/PypeIt) = general tool for ground-based spectroscopic extraction and reduction, supports lots of instruments, interface seems a little rigid
- [Eureka](https://github.com/kevin218/Eureka/) = general tool for extracting and fitting time-series data with JWST instruments (and possibly also Hubble and ground-based?)
- [tshirt](https://github.com/eas342/tshirt) = Everett Schlawin's tools for photometry and spectroscopy, mostly with space-based data; seems to produce great results and be quietly great
- [Tiberius](https://github.com/JamesKirk11/Tiberius) = James Kirk's tools for ground-based and JWST spectral extraction
- [HSTaXe](https://github.com/spacetelescope/hstaxe) = slitless spectroscopy tools for Hubble (particularly WFC3) = might be helpful once we branch into slitless ground-based spectra
- [prose](https://github.com/lgrcia/prose) = tool for image reduction and photometry, has a nice interface
- [mosasaurus](https://github.com/zkbt/mosasaurus) = Zach + Hannah Diamond-Lowe's kludgy old code; we can/should steal whatever code snippts we want from here 

### 📓 What data did we gather? 

Hopefully our observing logs are perfect records of every exposure we gathered and the weather conditions throughout the night. Just in case they're not, it might be helpful to have an automatic method to create a tabular summary of exposures (filename, time, target, airmass, disperser, exposure time), a visual summary of the night (flip-book of well-scaled images from the spectrograph and the guider). `ccdproc` has a `ImageFileCollection` that can help scan quickly through lots of image headers.

#### Going forward from this step, we might:
- take a list of filenames, open the header of each, compiled useful information into an astropy `Table` 
- take a list of filenames and make an animation displaying all the images in order, with some diagnostic information

#### Going backward from this step, we might:
- *(once all the other simulation steps below are complete)* start from a `Table` of image types, produce a full simulated night of science and calibration images

## 🌧 How many photons did the detector record? 

Images are typically read off in kind of arbitrary units, often called either DN ("data number) or ADU ("analog-to-digital units"). The readout electronics also have a bias, meaning a baseline level from which they start their counting. We want to go from the raw images downloaded from the detector into something easier to work with for statistics and analyis = the estimated number of photons that each pixel recorded. `ccdproc` has useful tools for managing image files and uncertainties and for approaching bias subraction and gain correction, but it might also be easiest to work directly with the data as `numpy` arrays or `astropy` Quantities.

#### Going forward in this step, we might:
- use multiple bias images to make a median bias calibration image and estimate the read noise
- use the median bias and/or the overscan pixels to subtract bias from each image (different for each amplifier)
- trim off the overscan pixels because they're no longer necessary
- multiply by the gain ([0.6 e-/DN](https://www.apo.nmsu.edu/arc35m/Instruments/KOSMOS/userguide.html)) to convert to electrons
- create an image of the uncertainty in each pixel due to read noise and $\sqrt{N}$ photon noise 

#### Going backward in this step, we might:
- start from an image of illuminated pixels (in units of photoelectrons), stitch on overscan pixels, convert to DN, add bias level, add read noise

#### To test/explain this step, we might:
- explain in friendly words what's happening
- visualize the individual bias frames as an animation, along with the median 
- plot histograms of bias pixels along with a Gaussian curve at the estimated read noise
- confirm that pixels in a series of similar images varies by (approximately!) the estimated pixel uncertainties
- test that we recover the right answer from a simulated image

## 🫓 How sensitive are the detector pixels?

At the telescope we take flat-field exposures to try to estimate variations in the sensitivity of pixels across the detector. 


#### Going forward in this step, we might:
- use multiple quartz-lamp flat-field images to make a median, unnormalized, flat-field image 
- identify a mask of which pixels are actually illuminated by the dispersed spectrograph slit
- estimate pixel-to-pixel variations by filtering out large-scale features in the median flat due to the quartz lamp Planck spectrum and the spectrograph sensitivity; this sensitivity map represents the variations a star's spectrum will see if it wobbles slightly in the slit

#### Going backward in this step, we might:
- start from an image of photons hitting the pixels and multiply by the pixel-to-pixel sensitivity to introduce defects in the image

#### To test/explain this step, we might:
- explain in friendly words what's happening
- visualize the individual flat frames as an animation, along with the median 
- visualize the unnormalized flat-field frame, the smoothed large-scale features being removed, and the pixel-to-pixel sensitivity
- test that we recover the right answer from a simulated image

## 🖋 How is the light of a star positioned on the detector? 

The disperser light of a start gets spread out in a smooth arc across the detector, often called the "trace". `specreduce` has some useful tools for this step that we should definitely use!

#### Going forward in this step, we might:
- define a reference point for each star as one of the following:
    - the (x,y) position of where the spectral trace intersects with a line drawn in the spatial direction, about midway along the wavelength axis [= probably the simplest for long slit data where no other undispersed images were taken]
    - the (x,y) position of the star in an undispersed image, either through the slit or not [= probably most useful for multiobject or slitless spectrographs, particularly where stars may start or stop at different locations along the wavelength direction]
    - (possibly others as needed)
- extract a subarray of a particular window size around that 
- estimate spatial centroids and widths along the star's trace to figure out where the center at each wavelength is (either through flux-weighted moments or through analytic profile fits)
- fit a smooth polynomial curve to the spatial centroids (wavelength pixel as the independent, centroid position as dependent) and spatial widths (wavelength pixel as independent, spatial width as dependent)

#### Going backward in this step, we might:
- start from a star's (x,y) reference point and a function describing the trace position and width, and create an image showing the trace of the star along the detector

#### To test/explain this step, we might:
- explain in friendly words what's happening
- make a well-scaled imshow of a science image, with the following overplotted:
    - the per-pixel centroids and widths
    - the window being considered for estimating centroids along the trace
    - the smooth fitted curves showing the centroid and width
- test that we recover the right answer from a simulated image

## 🌄 How many photons come from the sky? 

Slit spectrographs allow light from both a star and the surrounding glowing sky to be recorded on the spectrograph. By estimating the flux at each wavelength from the sky, we can remove it from our extracted stellar spectra. `specreduce` has some useful tools for this step. `specutils` defines [`Spectrum1D` objects](https://specutils.readthedocs.io/en/stable/spectrum1d.html), which will be returned by many `specreduce` tools, so we should take some time to understand them.

#### Going forward in this step, we might:
- for some spectral trace function (defined above), define sky estimation windows on either side of the stellar spectrum, using one of the following approaches:
    - set some fix distance from the star and width of the sky aperture [= easier to automate and probably OK for isolated and well-centered stars]
    - interactively set regions, ideally on either side of the star [= easier to do visually; easier to avoid nearby stars, scattered light, edge effects]
- estimate the sky flux for each wavelength pixel within the sky estimate region, using one of the following approaches:
    - take the median along the spatial direction [= robust to outliers but likely to fail for strongly curve traces or non-uniform illumination]
    - do an outlier-resistant polynomial fit to the flux along the spatial direction [= slightly more complicated but better for (slightly) curved traces and non-uniform illumination patterns]
- generate an image of the photon noise uncertainty from the sky flux on each pixel

#### Going backward in this step, we might:
- start from a spectrum of the sky background and an assumption for how that flux is distributed spatially, generate an image of the sky flux dispersed on the detector

#### To test/explain this step, we might:
- explain in friendly words what's happening
- make a well-scaled imshow with three panels, all with the sky estimation windows overplotted:
    - the (bias/flat-calibrated) science image 
    - the estimate of flux from the sky on the detector
    - the science image with the sky subtracted 
- test that we recover the right answer from a simulated image

## 🌠 How many photons come from the star? 

Once we've calibrated detector effects and defined a trace where light from the star falls, we can extract the stellar spectrum. This step often happens at about the same time as sky estimate, since the light falling on the star's pixels naturally includes both stellar and sky light. `specreduce` has some useful tools for this step.

#### Going forward in this step, we might:
- for some spectral trace function (defined above), extract the flux coming the star, using one of the following approaches:
    - define a boxcar aperture of a particular width centered on the trace  and sum the flux from the sky-subtracted science image along the spatial direction [= the simplest and best place to start]
    - use an estimate of the trace profile to perform a flux-weighted extraction (sometimes call ``optimal extraction''), fitting for the amplitude of the stellar trace [= useful for some purposes and particularly faint stars, but probably best to wait until everything else is working smoothly and easily]
- populate arrays with one element for each wavelength, capturing:
    - star flux
    - total flux uncertainty
    - sky flux 
    - uncertainty from star photons
    - uncertainty from sky photons
    - uncertainty from read noise
    - diagnostics (spatial offset, spatial width, peak pixel, ...)

#### Going backward in this step, we might:
- start from a spectrum of a star and a trace (defining centroids and widths), produce an image of the star dispersed on the detector (with no sky background)
- produce an image including both sky (spread out along the slit) and star (confined to a narrow Gaussian profile) flux

#### To test/explain this step, we might:
- explain in friendly words what's happening
- make a two-panel plot, with wavelength pixel on the x-axis
    - plot the star flux, the star + sky flux, the sky flux (in different colors)
    - plot the uncertainties in the overall extracted stellar spectrum, along with individual components contributed by read noise, sky photon noise, star photon noise
- test that we recover the right answer from simulated images (both with and without sky)

## 🌊 How do wavelength pixels relate to real wavelengths? 

Up to this point, we've measure everything in the wavelength direction in units of "wavelength pixels", not actual wavelength in $n\mathrm{m}$ or $\mu \mathrm{m}$. At the telescope, we took arc lamp calibration data to try to figure out the "wavelength calibration", meaning a function that relates wavelength pixel to real wavelength. `specreduce` has some useful tools for this step.

#### Going forward in this step, we might:
- use a star's trace to extract spectra for all arc lamps, at approximately the same location as where the star appears
- find the wavelength pixel locations of all the peaks in the extracted arc lamp spectra
- produce tables matching the wavelength pixel location of the peaks to the known wavelengths of the arc lamp emission lines; merge tables of all calibration lamps into one table
- fit a polynommial function to go from wavelength pixel location to real wavelengths
- populate an array of wavelengths associated with each flux/uncertainty value in this spectrum

#### Going backward in this step, we might:
- start from a list of known arc lamp wavelengths/strengths and a wavelength calibration function, produce a simulated 1D spectra on the wavelength pixel grid 
- start from a simulated 1D spectrum on the wavelength pixels to produce simulated 2D images (just zoomed in near the star) of the arc lamp spectra

#### To test/explain this step, we might:
- explain in friendly words what's happening
- compare a simulated 1D arc lamp spectrum to an observed one
- compare a simulated 2D arc lamp image to a real one
- test that we recover the right answer from simulated images

## 🤖 How do we automate for many exposures? 

For transit observations, we'll want to extract many spectra for multiple stars from many images, so having a way to automate the process will be very helpful. This is mostly stitching together the tools we'll have set up above inside of a loop over lots of exposures. 

#### Going forward in this step, we might:
- generate a mean science image combinining all science exposures for a particular pointing together; use this to define a shared trace and wavelength calibration to be used for all individual exposures
- create a `Rainbow` object for each star and populate it as follows:
    - loop through science exposures, extract stellar spectra, store all spectra (and all uncertainties and diagnostics) for each exposure as a columns in a `fluxlike` arrays
    - store observation times and all other quantities that change just with time (and not with wavelength) in `timelike` arrays 
    - store shared wavelengths and all other quantities that chabnge just with wavelength (and not with time) in `wavelike` arrays


#### Going backward in this step, we might:
- start from a simulated `Rainbow` and an estimate of the spectrograph throughput and wavelength calibration, generate series of science exposures

#### To test/explain this step, we might:
- explain in friendly words what's happening
- generate an animation flipping through time and showing spectra (and lots of diagnostic information) for all stars
- generate an animation flipping through wavelength and showing light curves (and lots of diagnostic information) for all stars
- test that we recover the right answer from simulated images